In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:

# Load the dataset
dataset=pd.read_csv('MultiLanguageTrainDataset.csv')
print(list(dataset.columns.values))


dataset.head()
# Extract text and labels
# texts = dataset['tweet']
# labels = dataset['class']

['Unnamed: 0', 'text', 'label', 'language']


,Unnamed: 0,text,label,language
0,0,لك من جهة ان شاء بضل حرقة بقلبك وبلاطه ع صدرك ...,0.0,1
1,1,انجاز جديد للعهدجبران باسيل,0.0,1
2,2,قلبه كبير وئام وديموقراطي عكس باسيل هههه,0.0,1
3,3,ولك العما بقلبك عمي يا اوطى مخلوق عوج الارض رو...,1.0,1
4,4,يسعد صباحك يا قائد,0.0,1


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout


ModuleNotFoundError: No module named 'tensorflow'

In [21]:
english_data = dataset[dataset['language'] == 2]
english_data.head()

english_data.shape

(53146, 4)

In [22]:
test = pd.read_csv('English_test.csv')
test.head()

,Unnamed: 0,text,label
0,141059,RT @warreports: 8 Pakistanis of #IRGC 's Zeyna...,1.0
1,6108,gronk tide pod commercial retarded,1.0
2,108845,@sfrantzman @pmakela1 what they found in Syria...,0.0
3,126712,After Pakistani Taliban announcement of their ...,0.0
4,127530,Good luck getting our attention this week ISIS...,0.0


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# Import Tokenizer and pad_sequences from tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Load the dataset
dataset = pd.read_csv('MultiLanguageTrainDataset.csv')

# Filter English data
english_data = dataset[dataset['language'] == 2]

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    english_data['text'].astype(str),
    english_data['label'],
    test_size=0.2,
    random_state=42
)

# Tokenization and preprocessing
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_data = pad_sequences(val_sequences, maxlen=max_sequence_length)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=5, batch_size=64)

# Load the test data
test = pd.read_csv('English_test.csv')
test_texts = test['text'].astype(str)
test_labels = test['label']

# Preprocess the test data
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
predictions = model.predict(test_data)
predictions_binary = (predictions > 0.5).astype(int)

# Classification report
print(classification_report(test_labels, predictions_binary))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 91s 134ms/step - accuracy: 0.8650 - loss: 0.3023 - val_accuracy: 0.9463 - val_loss: 0.1460
Epoch 2/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 129s 116ms/step - accuracy: 0.9685 - loss: 0.0939 - val_accuracy: 0.9463 - val_loss: 0.1441
Epoch 3/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 70s 97ms/step - accuracy: 0.9852 - loss: 0.0476 - val_accuracy: 0.9437 - val_loss: 0.1694
Epoch 4/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 80s 94ms/step - accuracy: 0.9925 - loss: 0.0245 - val_accuracy: 0.9439 - val_loss: 0.2076
Epoch 5/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 82s 95ms/step - accuracy: 0.9958 - loss: 0.0134 - val_accuracy: 0.9441 - val_loss: 0.2291
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9447 - loss: 0.2198
Test Loss: 0.2067
Test Accuracy: 0.9475
185/185 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.95      3192
         1.0       0.94      0.95      0.94      2714

    accuracy                  

In [29]:
# Load the dataset
dataset = pd.read_csv('MultiLanguageTrainDataset.csv')
print("english train dataset")
dataset.head()

# Filter English data
english_data = dataset[dataset['language'] == 2]
print("english filtered dataset")
dataset.head()

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    english_data['text'].astype(str),
    english_data['label'],
    test_size=0.2,
    random_state=42
)

# Tokenization and preprocessing
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_data = pad_sequences(val_sequences, maxlen=max_sequence_length)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=5, batch_size=64)

# Load the test data
test = pd.read_csv('English_test.csv')
test_texts = test['text'].astype(str)
test_labels = test['label']

# Preprocess the test data
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
predictions = model.predict(test_data)
predictions_binary = (predictions > 0.5).astype(int)

# Classification report
print(classification_report(test_labels, predictions_binary))


english train dataset
english filtered dataset
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


665/665 ━━━━━━━━━━━━━━━━━━━━ 67s 98ms/step - accuracy: 0.8701 - loss: 0.3056 - val_accuracy: 0.9457 - val_loss: 0.1481
Epoch 2/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 63s 95ms/step - accuracy: 0.9673 - loss: 0.0936 - val_accuracy: 0.9460 - val_loss: 0.1436
Epoch 3/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 82s 95ms/step - accuracy: 0.9858 - loss: 0.0491 - val_accuracy: 0.9429 - val_loss: 0.1696
Epoch 4/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 82s 95ms/step - accuracy: 0.9929 - loss: 0.0244 - val_accuracy: 0.9431 - val_loss: 0.1970
Epoch 5/5
665/665 ━━━━━━━━━━━━━━━━━━━━ 62s 94ms/step - accuracy: 0.9965 - loss: 0.0144 - val_accuracy: 0.9410 - val_loss: 0.2240
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9394 - loss: 0.2206
Test Loss: 0.2058
Test Accuracy: 0.9448
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      3192
         1.0       0.94      0.94      0.94      2714

    accuracy                           0.94

In [30]:
# Load the dataset
dataset = pd.read_csv('MultiLanguageTrainDataset.csv')

# Filter English data
french_data = dataset[dataset['language'] == 4]

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    french_data['text'].astype(str),
    french_data['label'],
    test_size=0.2,
    random_state=42
)

# Tokenization and preprocessing
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_data = pad_sequences(val_sequences, maxlen=max_sequence_length)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=5, batch_size=64)

# Load the test data
test = pd.read_csv('French_test.csv')
test_texts = test['text'].astype(str)
test_labels = test['label']

# Preprocess the test data
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
predictions = model.predict(test_data)
predictions_binary = (predictions > 0.5).astype(int)

# Classification report
print(classification_report(test_labels, predictions_binary))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.6861 - loss: 0.5919 - val_accuracy: 0.8328 - val_loss: 0.3987
Epoch 2/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - accuracy: 0.8869 - loss: 0.3074 - val_accuracy: 0.8333 - val_loss: 0.3838
Epoch 3/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 19s 87ms/step - accuracy: 0.9415 - loss: 0.1660 - val_accuracy: 0.8450 - val_loss: 0.4003
Epoch 4/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.9684 - loss: 0.0954 - val_accuracy: 0.8470 - val_loss: 0.4418
Epoch 5/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 21s 99ms/step - accuracy: 0.9777 - loss: 0.0638 - val_accuracy: 0.8470 - val_loss: 0.4855
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8236 - loss: 0.6016
Test Loss: 0.5768
Test Accuracy: 0.8323
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
              precision    recall  f1-score   support

         0.0       0.84      0.81      0.82       532
         1.0       0.82      0.86      0.84       565

    accuracy                           0.83    

In [31]:

# dataset = pd.read_csv('your_hindi_dataset.csv')
dataset = pd.read_csv('MultiLanguageTrainDataset.csv')

# Filter hindi data
hindi_data = dataset[dataset['language'] == 8]

# Split the dataset into training (80%) and testing (20%) sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Separate the text and labels for training and testing
train_texts = train_data['text'].astype(str)
train_labels = train_data['label']

test_texts = test_data['text'].astype(str)
test_labels = test_data['label']

# Tokenization and preprocessing
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=5, batch_size=64)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
predictions = model.predict(test_data)
predictions_binary = (predictions > 0.5).astype(int)

# Classification report
print(classification_report(test_labels, predictions_binary))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
2750/2750 ━━━━━━━━━━━━━━━━━━━━ 248s 90ms/step - accuracy: 0.7447 - loss: 0.4934
Epoch 2/5
2750/2750 ━━━━━━━━━━━━━━━━━━━━ 262s 90ms/step - accuracy: 0.8310 - loss: 0.3594
Epoch 3/5
2750/2750 ━━━━━━━━━━━━━━━━━━━━ 248s 90ms/step - accuracy: 0.8868 - loss: 0.2585
Epoch 4/5
2750/2750 ━━━━━━━━━━━━━━━━━━━━ 247s 90ms/step - accuracy: 0.9331 - loss: 0.1617
Epoch 5/5
2750/2750 ━━━━━━━━━━━━━━━━━━━━ 262s 90ms/step - accuracy: 0.9536 - loss: 0.1101
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.7881 - loss: 0.6608
Test Loss: 0.6645
Test Accuracy: 0.7898
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step
              precision    recall  f1-score   support

         0.0       0.82      0.84      0.83     27060
         1.0       0.74      0.70      0.72     16937

    accuracy                           0.79     43997
   macro avg       0.78      0.77      0.78     43997
weighted avg       0.79      0.79      0.79     43997

